<a href="https://colab.research.google.com/github/astroviki/NumericalMethods/blob/main/radiativetransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy import constants as const


# Jednoduchý model hvězdné atmosféry

Příklad včetně postup výpočtu byl převzat z knihy [P. Hellings *Astrophysics with a PC*](https://www.amazon.com/gp/product/B010WFIZTK/ref=dbs_a_def_rwt_hsch_vapi_taft_p1_i0). Jedná se o velmi zjednodušený model hvězdné atmosféry, určené dvěma volnými parametry: Efektivní teplotou na povrchu $T_{\rm eff}$ a gravitačním zrychlením $g_s$ na povrchu hvězdy.

V rámci modelu uvažujeme pro zjednodušení uvažujeme chemické složení stejné jako v naší Galaxii. Na chemickém složení závisí především absorbční koeficient $\kappa$ a výpočet střední molekoulové hmotnosti $\mu$.



Definujeme si funkce pro výpočet jednotlivých veličin, charakterizujících astmosféru: Teplotu $T$[K], zářivé zrychlení $g_{\rm rad}$, tlak plynu $P_{\rm g}$, tlak záření $P_{\rm rad}$

In [11]:
def temperature(tau):
  q = 0.7104-0.1331*np.exp(-3.4488*tau)
  return (3.0/4.0*Teff**4*(tau+q))**0.25

def rad_press(T):
  return 1.0/3.0*a*T**4.0

def eqstate(T):
  return Pg*mu/(T*R)

def absorb(rho,T):
  return 1.984e24*rho*T**(-3.5)

def rad_accel(kappa,tau):
  return kappa/4.0*a*Teff**4*(1.0+0.459**np.exp(-3.4488*tau))

def gefect(kappa,tau):
  return gs-rad_accel(kappa,tau)

def density(T,Pg):
  return mu*Pg/(R*T)



###Střední molekulová hmotnost 

| T/log g | 5000  | 6000  | 7000  | 8000  | 10000 | 12000 | 14000 | 16000 |
|---------|-------|-------|-------|-------|-------|-------|-------|-------|
| 4       | 1.289 | 1.287 | 1.285 | 1.277 | 1.048 | 0.724 | 0.682 | 0.672 |
| 3       | 1.287 | 1.285 | 1.284 | 1.266 | 0.868 | 0.688 | 0.673 | 0.659 |
| 2       | 1.285 | 1.285 | 1.280 | 1.232 | 0.738 | 0.675 |   -   |   -   |


In [12]:
# Physical constants
R = 8.314e7
a = 7.56464e-15
G = 6.673e-8
Teff = 10000
mu = 1.048
rhoin = 1e-13 
gs = 10**(4)
N = 32

tau = np.empty(N)
T = np.empty(N)
Pg = np.empty(N)
Pr = np.empty(N)
rho = np.empty(N)
kappa = np.empty(N)
gr = np.empty(N)
ge = np.empty(N)
z = np.zeros(N)

tau[0] = 0.0
tau[1] = 0.001

for i in range(N-2):
  tau[i+2]=1.25*tau[i+1]

dtau = 0.25*tau
dtau[0] = 0.001

def initial(tau):
  T = temperature(tau)
  Pr = rad_press(T)
  Pg = rhoin*R*T/mu
  kappa = absorb(rhoin,T)
  gr = rad_accel(kappa,tau)
  ge = gefect(kappa,tau)
  return T,Pr,Pg,kappa,gr,ge

T[0],Pr[0],Pg[0],kappa[0],gr[0],ge[0] = initial(tau[0])

rho[0] = rhoin


In [22]:
for i in range(N-1):

  # half_point
  tau_mid = tau[i]+0.5*dtau[i]
  T_mid = temperature(tau_mid)
  Pg_mid = Pg[i]+0.5*dtau[i]*ge[i]/kappa[i]
  rho_mid = density(T_mid,Pg_mid)
  kappa_mid = absorb(rho_mid,T_mid)
  ge_mid = gefect(kappa_mid,tau_mid)

  # full_point
  tau[i+1] = tau[i]+dtau[i]
  T[i+1] = temperature(tau[i+1])
  Pg[i+1] = Pg[i]+dtau[i]*ge_mid/kappa_mid
  rho[i+1] = density(T[i+1],Pg[i+1])
  kappa[i+1] = absorb(rho[i+1],T[i+1])
  ge[i+1] = gefect(kappa[i+1],tau[i+1])

  dz = dtau[i]//(kappa_mid*rho_mid)
  z[i+1] = z[i] + dz
  Pr[i+1] = rad_press(T[i+1])


In [23]:
for i in range(N):
  print("%.2f" % Pg[i])

0.06
0.17
0.49
1.45
4.16
9.88
16.38
22.35
28.18
34.14
40.40
47.10
54.36
62.31
71.06
80.77
91.59
103.71
117.34
132.74
150.23
170.18
193.07
219.47
250.10
285.85
327.82
377.40
436.30
506.71
591.36
693.76


In [6]:
const.G.cgs*const.R_sun.cgs**2/const.M_sun.cgs*const.M_sun.cgs**2/const.R_sun.cgs**5


<Quantity 3.94135564e-07 1 / s2>